In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [29]:
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [30]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [31]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [32]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)


In [37]:
query = "Who is Juniper?"
docs = db.similarity_search(query)

In [39]:
print(wrap_text_preserve_newlines(str(docs[0].page_content)))

One day, while traveling through time, Juniper and her team stumbled upon a shocking discovery. They had been
traveling through a fabricated reality created by an unknown entity. The reality they had known for so long
was nothing more than a simulation.

This revelation shook Juniper and her team to the core. They had been living in a world of lies and deceit,
but they refused to give up. They continued their journey through time, hoping to find a way to break free of
the simulation and restore reality to its true form.

As they traveled further back in time, they discovered something even more disturbing. The entity that had
created the simulation was none other than a future version of Juniper herself. She had traveled back in time
to create the simulation and change the course of history.


In [40]:
def query_formatter(docs, query):
    return f""" Use only below 'Context' and find the answer to the 'Question'. Be precise and to the point while answering.
    'Context: {docs}'
    'Question: {query}'"""

In [41]:
query_formatter(docs=docs, query=query)

" Use only below 'Context' and find the answer to the 'Question'. Be precise and to the point while answering.\n    'Context: [Document(page_content='One day, while traveling through time, Juniper and her team stumbled upon a shocking discovery. They had been traveling through a fabricated reality created by an unknown entity. The reality they had known for so long was nothing more than a simulation.\\n\\nThis revelation shook Juniper and her team to the core. They had been living in a world of lies and deceit, but they refused to give up. They continued their journey through time, hoping to find a way to break free of the simulation and restore reality to its true form.\\n\\nAs they traveled further back in time, they discovered something even more disturbing. The entity that had created the simulation was none other than a future version of Juniper herself. She had traveled back in time to create the simulation and change the course of history.', metadata={'source': './state_of_the_u

In [42]:
import requests
import time

url = 'https://www.quora.com/poe_api/gql_POST'

headers = {
    'Host': 'www.quora.com',
    'Accept': '*/*',
    'apollographql-client-version': '1.1.6-65',
    'Accept-Language': 'en-US,en;q=0.9',
    'User-Agent': 'Poe 1.1.6 rv:65 env:prod (iPhone14,2; iOS 16.2; en_US)',
    'apollographql-client-name': 'com.quora.app.Experts-apollo-ios',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
}

def set_auth(key, value):
    headers[key] = value

def load_chat_id_map(bot="a2"):
    data = {
        'operationName': 'ChatViewQuery',
        'query': 'query ChatViewQuery($bot: String!) {\n  chatOfBot(bot: $bot) {\n    __typename\n    ...ChatFragment\n  }\n}\nfragment ChatFragment on Chat {\n  __typename\n  id\n  chatId\n  defaultBotNickname\n  shouldShowDisclaimer\n}',
        'variables': {
            'bot': bot
        }
    }    
    response = requests.post(url, headers=headers, json=data)
    return response.json()['data']['chatOfBot']['chatId']

def send_message(message,bot="a2",chat_id=""):
    data = {
    "operationName": "AddHumanMessageMutation",
    "query": "mutation AddHumanMessageMutation($chatId: BigInt!, $bot: String!, $query: String!, $source: MessageSource, $withChatBreak: Boolean! = false) {\n  messageCreate(\n    chatId: $chatId\n    bot: $bot\n    query: $query\n    source: $source\n    withChatBreak: $withChatBreak\n  ) {\n    __typename\n    message {\n      __typename\n      ...MessageFragment\n      chat {\n        __typename\n        id\n        shouldShowDisclaimer\n      }\n    }\n    chatBreak {\n      __typename\n      ...MessageFragment\n    }\n  }\n}\nfragment MessageFragment on Message {\n  id\n  __typename\n  messageId\n  text\n  linkifiedText\n  authorNickname\n  state\n  vote\n  voteReason\n  creationTime\n  suggestedReplies\n}",
    "variables": {
        "bot": bot,
        "chatId": chat_id,
        "query": message,
        "source": None,
        "withChatBreak": False
    }
}
    _ = requests.post(url, headers=headers, json=data)

def clear_context(chatid):
    data = {
        "operationName": "AddMessageBreakMutation",
        "query": "mutation AddMessageBreakMutation($chatId: BigInt!) {\n  messageBreakCreate(chatId: $chatId) {\n    __typename\n    message {\n      __typename\n      ...MessageFragment\n    }\n  }\n}\nfragment MessageFragment on Message {\n  id\n  __typename\n  messageId\n  text\n  linkifiedText\n  authorNickname\n  state\n  vote\n  voteReason\n  creationTime\n  suggestedReplies\n}",
        "variables": {
            "chatId": chatid
        }
    }    
    _ = requests.post(url, headers=headers, json=data)

def get_latest_message(bot):
    data = {
        "operationName": "ChatPaginationQuery",
        "query": "query ChatPaginationQuery($bot: String!, $before: String, $last: Int! = 10) {\n  chatOfBot(bot: $bot) {\n    id\n    __typename\n    messagesConnection(before: $before, last: $last) {\n      __typename\n      pageInfo {\n        __typename\n        hasPreviousPage\n      }\n      edges {\n        __typename\n        node {\n          __typename\n          ...MessageFragment\n        }\n      }\n    }\n  }\n}\nfragment MessageFragment on Message {\n  id\n  __typename\n  messageId\n  text\n  linkifiedText\n  authorNickname\n  state\n  vote\n  voteReason\n  creationTime\n}",
        "variables": {
            "before": None,
            "bot": bot,
            "last": 1
        }
    } 
    author_nickname = ""
    state = "incomplete"
    while True:
        time.sleep(2)
        response = requests.post(url, headers=headers, json=data)
        response_json = response.json()
        text = response_json['data']['chatOfBot']['messagesConnection']['edges'][-1]['node']['text']
        state = response_json['data']['chatOfBot']['messagesConnection']['edges'][-1]['node']['state']
        author_nickname = response_json['data']['chatOfBot']['messagesConnection']['edges'][-1]['node']['authorNickname']
        if author_nickname==bot and state=='complete':
            break
    return text


In [43]:
#from POE import load_chat_id_map, clear_context, send_message, get_latest_message, set_auth

#Auth
set_auth('Quora-Formkey','f72407929b50a310a71f0bfab73594cc')
set_auth('Cookie','m-b=fo4BP-aQf5SXPYGN7sDb3A==')
#---------------------------------------------------------------------------
# print("Who do you want to talk to?")
# print("1. Sage - OpenAI (capybara)")
# print("2. GPT-4 - OpenAI (beaver)")
# print("3. Claude+ - Anthropic (a2_2)")
# print("4. Claude - Anthropic (a2)")
# print("5. ChatGPT - OpenAI (chinchilla)")
# print("6. Dragonfly - OpenAI (nutria)")
#---------------------------------------------------------------------------
# option = input("Please enter your choice : ")
bots = {1:'capybara', 2:'beaver', 3:'a2_2', 4:'a2', 5:'chinchilla', 6:'nutria'}
bot = bots[int(1)]
print("The selected bot is : ", bot)
#---------------------------------------------------------------------------
chat_id = load_chat_id_map(bot)
clear_context(chat_id)
print("Context is now cleared")

message = query_formatter(docs=docs, query=query)
send_message(message,bot,chat_id)
reply = get_latest_message(bot)
print(f"{bot} : {reply}")

# while True:
#     message = input("Human : ")
#     if message =="!clear":
#         clear_context(chat_id)
#         print("Context is now cleared")
#         continue
#     if message =="!break":
#         break
#     send_message(message,bot,chat_id)
#     reply = get_latest_message(bot)
#     print(f"{bot} : {reply}")

The selected bot is :  capybara
Context is now cleared
capybara : Juniper is a person who traveled through time with her team and discovered that they had been living in a simulated reality created by a future version of herself. She also created a time machine and traveled through different eras, but realized that their actions were having unintended consequences. Juniper sacrificed herself to destroy the simulation and restore reality to its true form, but the team ended up in a barren wasteland and had to rebuild the world.
